# 事故情報（accidentインデックス）の事故内容に類似する事故を検索(anzenインデックス)し、その状況や原因、対策をサジェストする。

## 1. まず事故情報（accidentインデックス）から類似検索したい事故を探す
* ここでは単純なキーワード検索で事故情報を検索する。

In [203]:
import elasticsearch, json
import pandas as pd

HOST = "elasticsearch"
PORT = "9200"
INDEX_NAME = "accident"
SCROLL_TIME = '1m'
SCROLL_SIZE = 10
pd.set_option("display.max_colwidth", 100)

search_word = "カバー"

client = elasticsearch.Elasticsearch(HOST + ":" + PORT)

results = []
scroll_results = client.search(
    index = INDEX_NAME,
    scroll = SCROLL_TIME,
    size = SCROLL_SIZE,
    body = {"query": { "match": {"sentence":search_word} }}
)

sid = scroll_results['_scroll_id']
scroll_results = scroll_results['hits']['hits']
scroll_size = len(scroll_results)

for scroll_result in scroll_results:
    results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "sentence":scroll_result['_source']['sentence']})

while scroll_size > 0:
    scroll_results = client.scroll(
        scroll_id = sid,
        scroll = SCROLL_TIME )
    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)
    
    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "sentence":scroll_result['_source']['sentence']})

df = pd.DataFrame(results)

# ここでは最初の10件のみ表示させておく
df = df.head(10)
df_style = df.style.set_properties(subset=["sentence"], **{'text-align': 'left'})
df_style

,id,category,sentence
0,vIxJTHABx8pAyv6FC09b,製造業,レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
1,MoxKTHABx8pAyv6F2XRM,建設業,杭打ち機のメンテナンス作業中、中間振れ止めカバーが落下し、下部振れ止めカバーとの間に挟まれた。
2,3YxHTHABx8pAyv6FBCby,建設業,めっき槽に付属する配管の塗装作業中、めっき槽上に取り付けてあったカバーが外されていたが、被災者はカバーがあると勘違いしてカバーの上を渡ろうとして、めっき槽の中に落ちて被災した。
3,U4xJTHABx8pAyv6FYVbT,製造業,フェルト製品の製造作業中において、綿を送り出すための装置の下部カバー内で綿が詰まったため、被災者が詰まった綿をカバー下部から取り出していた所、綿が一度に大量に落ち、カバーから引き抜いた際にカバー下部にある脱水機のシュート部で回転していた綿に引き込まれ、脱水機の壁面で強打した。
4,5YxGTHABx8pAyv6FHxXN,建設業,天井クレーン用走行レールの点検作業後、梯子で地上に降りる途中、両手に持っていた車輪カバーのカバーが開いたため体勢が崩れ、約６．３ｍ下の床に転落した。
5,14xKTHABx8pAyv6FLWaP,建設業,アース・オーガーで穴明け作業をするためドリルパイプの位置調整をしていたときに、土砂飛散防止用カバーを吊っていたワイヤーロープが切断したためカバーが落下して当った。
6,AIxKTHABx8pAyv6F1nSc,製造業,変圧器用タンクカバーの溶接作業に際し、設置位置の台を修正するため、カバー（重さ3.5トン）を天井クレーンでつり上げたところ、ワイヤロープ（径9.3mm）が破断し、当該カバーが倒れ、その下敷きとなった。
7,xoxFTHABx8pAyv6FJwGv,製造業,発電設備であるガスタービンの鉄製防音カバー（天井部分と側壁2面がボルトにより結合されたもので、質量約1.1t）内側のシート張りと解体作業中、倒壊防止のための補強が不十分であったためカバー倒壊し、中で作業していた2名のうち1名がカバーの下敷きになり死亡し、１名が骨盤、脚を骨折した。
8,SYxGTHABx8pAyv6FTBn3,建設業,自転車用道路の舗装工事中、タイヤローラーを後進させていたところ、タイヤローラー座席の台座カバー取付部が腐食していたためカバーと座席が外れ、座席ごと車外へ転落した際タイヤローラーにひかれた。
9,BoxITHABx8pAyv6F3UxR,建設業,駐車場新設工事において、被災者はスタビライザー（路盤用機械）を運転して地面に散布したセメントを混合する作業を行なっていたが、機体のロータとロータ部分を覆うカバーの間に石がはさまったため、機械を停止させて運転席後方よりカバー内に入り、石を取り除く作業を行なっていたところ、カバーが落下し、はさまれた。


## 2. 上記の検索結果の中から、類似検索したい事故を指定する
* キーワード検索した結果の中から、類似検索したい事故を探す。
* 類似検索したい事故の一番左の列の番号を、target_idに設定する。
* 選択した事故の内容が表示される。

In [204]:
# 特定の行を参照する。0始まり
target_id = 0
target = df.loc[target_id]
print("以下の事故について、類似事象を抽出します。")
target

以下の事故について、類似事象を抽出します。


id                                                                                   vIxJTHABx8pAyv6FC09b
category                                                                                              製造業
sentence    レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
Name: 0, dtype: object

## 3. 上記で指定した事故に類似する事故・状況・原因・対策を検索し、トップ10を表示する

### 3-1. 指定した事故の文章ベクトルを取得する

In [238]:
# 対象の文章ベクトルを取得する。
id = target['id']
target_vector = client.search(
    index = "vector",
    body = {"query": { "match": {"doc_id":id} }}
)
target_vector = target_vector['hits']['hits'][0]['_source']['vector']

### 3-2. 類似検索のためのクエリ
* 事故の内容に関する類似検索なので、タイトルと状況のみを検索対象としている。

In [215]:
# 類似検索用のクエリ
# 事故内容に対して、タイトルと事象が類似する事例を抽出する。
script_query = {
    "script_score": {
        "query": {
            "bool" : {
                "should" : [ { 
                    "bool" : {
                        "must": [
                            {"term": { "category":"title"}},
                            {"exists":{"field": "vector"}}
                        ]
                    } 
                }, {
                    "bool": {
                        "must" : [
                            {"term": { "category":"situation"}},
                            {"exists":{"field": "vector"}}
                        ]
                    }
                }  ]
            }
        },
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['vector']) + 1.0",
            "params": {"query_vector": target_vector}
        }
    }
}

### 3-3. 類似するタイトルと状況を検索する
* 指定した事故に類似する、タイトルと状況のトップ10が抽出される。

In [239]:
# 類似する文書を検索する。ここでは、類似するタイトルと状況のみとなる。
responses = client.search(
    index="vector",
    body={
        "size": SCROLL_SIZE,
        "query": script_query,
        "_source": {"includes": ["_id", "doc_id", "category", "text_id", "text"]}
    }
)
responses = responses['hits']['hits']
anzen_doc_ids = []
for response in responses:
    text_id = response['_source']['text_id']
    anzen_doc_ids.append(text_id.split('_')[0])

# トップ10のドキュメント番号
anzen_doc_ids

['100581',
 '100893',
 '101061',
 '545',
 '100034',
 '621',
 '101543',
 '100392',
 '100893',
 '100636']

### 3-4. 類似事例について全項目を取得
* 上記で抽出したタイトル、状況について、該当する事例を取得し表示する。

#### 指定したdoc_idの事例（タイトル、状況、原因、対策）を検索する関数

In [227]:
# 指定したdoc_idの事例（タイトル、状況、原因、対策）を検索する。
def search_doc(doc_id):
    SCROLL_TIME = '1m'
    SCROLL_SIZE = 10
    results = []
    scroll_results = client.search(
        index = "anzen",
        scroll = SCROLL_TIME,
        size = SCROLL_SIZE,
        body = {
            "query": { "term": {"doc_id":doc_id} },
            "_source": {"includes": ["doc_id", "category", "text_id", "text"]}
        }
    )

    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)

    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "text_id":scroll_result["_source"]["text_id"],"text":scroll_result['_source']['text']})

    while scroll_size > 0:
        scroll_results = client.scroll(
            scroll_id = sid,
            scroll = SCROLL_TIME )
        sid = scroll_results['_scroll_id']
        scroll_results = scroll_results['hits']['hits']
        scroll_size = len(scroll_results)

        for scroll_result in scroll_results:
            results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['category'], "text_id":scroll_result["_source"]["text_id"],"text":scroll_result['_source']['text']})

    sorted_results = sorted(results, key=lambda x:x['text_id'])
    return sorted_results

### 結果を整形して出力する。

In [244]:
results = []
for anzen_doc_id in anzen_doc_ids:
    anzen_doc_lines = search_doc(anzen_doc_id)
    
    title = ([anzen_doc_line['text'] for anzen_doc_line in anzen_doc_lines  if anzen_doc_line['category'] == 'title'])
    situations = ([anzen_doc_line['text'] for anzen_doc_line in anzen_doc_lines  if anzen_doc_line['category'] == 'situation'])
    situations = "\n".join(situations)    
    causes = ([anzen_doc_line['text'] for anzen_doc_line in anzen_doc_lines  if anzen_doc_line['category'] == 'cause'])
    causes = "\n".join(causes)
    measures = ([anzen_doc_line['text'] for anzen_doc_line in anzen_doc_lines  if anzen_doc_line['category'] == 'measures'])
    measures = "\n".join(measures)
    
    line = {"件名":title[0], "状況:":situations, "原因":causes, "対策":measures}
    results.append(line)    
    
df = pd.DataFrame(results)
df_style = df.style.set_properties(subset=["件名", "状況", "原因", "対策"], **{'text-align': 'left'})
print("検索した事故内容\n", target)
df_style

検索した事故内容
 id                                                                                   vIxJTHABx8pAyv6FC09b
category                                                                                              製造業
sentence    レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
Name: 0, dtype: object


,件名,状況:,原因,対策
0,古いエアーバックを廃棄処分中に、使用した古タイヤごと吹き飛ばされて地面に激突,この災害は、乗用車販売営業所において古いエアバッグを廃棄処分する作業中に発生したものである。 業務改善室員AとBは事業の効率化の現場指導の一環として在庫整理しているうちに不用になったエアバッグ1個をみつけた。 災害発生当日、敷地内で、このエアバッグを電流により作動させてバッグを膨脹させたのちに廃棄する作業を開始した。Aはホイールを外した古タイヤの内側にエアバッグを自動車用電線を用いて固定し、その上にさらに3本の古タイヤを重ねた。この時、下から2本目と4本目のタイヤにはホイールが付いたままであった。次にエアバッグの作動用電気導線を1mほど離れたところにおいたバッテリーのところにいるBに渡し、重ねられたタイヤを上から体重を掛けて押さえ付けて、Bにバッテリーによる通電を命じた。Aの合図によりBがこの電気導線をバッテリーの端子に接続して、タイヤに背を向けて通電したところ、「ドスン」という音を聞いたので振り向くと、タイヤとともに宙に浮いたA が前頭部からアスファルト地面に激突するところであった。,この災害は、古いエアバッグを廃棄処分中、エアバッグ作動の衝撃で跳ね飛ばされたものであるが、その原因としては、次のようなことが考えられる。 1 タイヤの積み方が悪かったこと エアバッグを安全に膨張させることのできる体積は、ホイールが付いていないタイヤを地面に4本積み重ね、5本目にホイールが付いたタイヤを積み重ねたとき一番上のタイヤのホイールが蓋となり地面が底となる高さとタイヤ内径で定まる空間が必要となるが、鉄製ホイールが付いたタイヤを2 本目に重ねたため、2本目のタイヤのホイールが蓋となりタイヤの内側空間の体積はエアバッグの作動して膨張した体積より小さくなり、安全に膨張させるためには、不十分である。 この状態でエアバッグを膨張させたため、2本目のタイヤが持ち上げられ、この力は膨張時に瞬間的に発生するため、タイヤを上から押さえていたAを跳ね飛ばした。 2 積み重ねたタイヤを固縛しなかったこと 3 廃棄作業を経験のない者にさせたこと 4 エアーバックの廃棄作業の危険性について認識がなく、安全教育を実施していなかったこと,同種災害を防止するためには、次のような対策が必要であると考えられる。 1 エアバッグ単体での廃棄作業手順を十分検討して作成すること 乗用車用のエアバッグは運転席エアバッグ、助手席エアバッグ、サイドエアバッグの3種類があり、その用途に応じて膨張の仕方が異なっているので、それぞれにエアバッグの種類に適した廃棄作業手順を定める。また、作業手順はホイールの付いていないタイヤを積み重ねたタイヤの内側の体積が、エアバッグが膨張したときの体積より大きくなるように本数とホイールの有無および順番を明確に定める。また、膨張したときの衝撃でタイヤが崩れないようにタイヤを紐で縛り、安全な距離を保って作動させるなど十分に検討して作成する。 2 エアバッグ廃棄作業者を選任すること 種類が多く、かつ、複雑な作業の上に手順を誤ると危険であるため、この作業に習熟した人に実施させることが必要である。 3 安全教育を実施し、安全管理を徹底すること
1,製材所のリングバーカーで皮剥ぎ作業中、丸太が飛び出し、作業者に激突,この災害は、製材所のリングバーカーで皮剥ぎ作業中の北洋材が突然左右に振れて飛び出し、バーカーの付近にいた作業者に激突したものである。 この工場は北洋材を輸入する商社の所有で、作業のみを行う子会社で製材作業が行われていたが、玉切り、皮剥ぎ作業は関連運送会社に委託していた。被災者Aは親会社からの出向者で工場長代行をしていた。 災害発生当日、午前8時の朝のミーテングで職長から作業指示があり、その後リングバーカー運転者Bは現場で待機していた。 午前9時頃、丸太がベルトコンベヤで送給されてきたので、Bは機械のスイッチをいれて皮剥ぎ作業を開始した。工場長代行Aはバーカー出口で、節のない材にスプレーで印しを付ける作業を始め、午前10時には休憩室で15分程休憩した後、現場にもどり作業を再開した。 午前10時20分頃、リングバーカー入口のカッター付近でバババンと大きな音がして、丸太がリングバーカーの押えローラーを押し上げて左右に振れた。運転者Bはリングバーカーオペレーター室で緊急停止スイッチを押して、リングバーカーを止め、リングバーカー入口まで様子を見に行くと、リングバーカーの押えローラー付近の通路でAが仰向けに倒れリングバーカーから脱落した丸太の下敷きになっていた。,この災害の原因としては、次のようなことが考えられる。 1 北洋材が皮剥ぎ中に丸太押さえローラーを押し上げてラインから脱落したこと。 リングバーカーは、丸太が送入される直前にローラーが降下して丸太が押さえられ、その状態でカッター部に送り込まれ、回転したカッターにより皮が剥がされる仕組みになっている。しかし、節があったり、株の張っている丸太は、カッターが正常に回転しないで丸太が暴れ、ローラーを押し上げてリングバーカーから脱落することがあった。 2 リングバーカー稼働中に丸太が押さえローラーを押し上げて、脱落するおそれのある危険場所に作業者が立ち入ったこと。 丸太が暴れてリングバーカーから脱落することが月1～2回程度あり、このローラー側の通路は稼働中はかなり危険性が高い場所であった。 3 節や株の張り出し部分などにカッターの刃が引っ掛かったこと。 丸太の暴走を防止する対策が講じられていなかった。 4 安全作業手順の定めがなく、作業の指揮命令系統が不明確であることなど安全管理体制が不備で、安全管理が不十分であったこと。,同種災害の防止のためには、次のような対策の徹底が必要である。 1 北洋材が皮剥ぎ中に丸太押さえローラーを押し上げてラインから脱落しない措置を講じること。 ローラー入り口にＶ字型カバーや柵等を設け丸太が暴れてもラインから脱落しないようにする。 2 丸太が押さえローラーを押し上げて、脱落するおそれのある場所には立入禁止の措置を講じること。 丸太押さえローラーの脇の通路には、稼働中には立ち入ることのできないようにドアを設け、稼働中には開閉できないようにするか、稼働中立入禁止の表示をすること。 3 節や株の張り出し部分などにカッターの刃が引っ掛かからないようにすること。 節等を除く前処理をしたり、刃の点検をするなど機械の整備を徹底する。 4 作業手順を作成して、安全管理体制を確立すること。 作業手順を作成し、指揮命令系統を明確にして、安全管理体制の確立を図るとともに、作業者の安全教育を実施し、安全管理を徹底する。 特に、複数の会社が協同して作業する場合には、相互の連絡を十分に行うことが大切である。
2,タイヤの成型作業中、成型機ドラムとガイドとにはさまれ死亡,この災害はトラック・バス用ラジアルタイヤ製造工程において、タイヤの成型作業中に発生したものである。 ラジアルタイヤの成型作業は、タイヤのチューブにあたるインナーライナーを成型機のドラムに貼り、その後、他の部材を順次貼っていくものである。 作業者Aは、タイヤ成型作業中、最初に成型機のドラムに貼り付けたインナーライナーに不良部分を見つけたので、不良インナーライナーを成型機から取り出そうとした。 Aは、操作盤が遠いため足元のマットスイッチを踏み成型機を停止させた後、成型機のドラム上に身を乗り出したところ、バランスを崩し、このためマットスイッチから足が離れてしまい、成型機が突然作動して、タイヤを押さえるガイドが降下し、ドラムとの間にはさまれて被災した。 タイヤ成型機には身体が危険範囲に入った場合に成型機を自動停止させる安全装置が設置されておらず、成型機の操作盤も作業位置から離れていた。 この工場では、非定常作業の作業手順書を作成しておらず、作業者への安全衛生教育も実施していなかった。,この災害の原因としては、次のことが考えられる。 1 身体が成型機にはさまれる危険性のあるインナーライナーの取出し等の作業について、自動停止させる安全装置が設置されていなかったこと 2 作業位置から操作できる位置に成型機の操作盤を設置していなかったこと 3 非定常作業の作業手順書（非定常作業では操作盤で機械を手動に切り替えて作業を行う）を作成していなかったこと 4 作業者への安全衛生教育を実施していなかったこと,同種災害を防止するためには、次のような対策が必要である。 1 インナーライナ－の取出し等の作業では、成型機を自動停止させる安全装置を設置すること 2 作業位置から操作できる位置に成型機の操作盤を移動し、作業者がいつでも成型機を停止できるようにすること 3 非定常作業の作業手順書を作成して、作業者に徹底させること 4 作業の実態を十分に把握した上で、作業の近道・省略行為等によるヒューマンエラーを防止するための安全衛生教育を実施すること
3,アセチレンガス半自動切断機に点火し、爆発,本災害は、造船所において、アセチレンガスを用いた半自動ガス切断機を使用しようとして、ライターで点火しようとしたところ、漏れ出ていたアセチレンガスに引火、爆発し、爆風で飛ばされた切断機が作業者に当たり、死亡したものである。 事故の発生したY造船は、鋼製の底引き漁船などの建造、修理を行っている造船所であり、Z工業は、その構内下請けとして鉄工関係の仕事を請け負っている。本災害の被害者Bは、Z工業の作業者である。 災害発生当